# Semantic segmentation test

---

**Author:** [rodoart](https://github.com/rodoart/)<br>
**Date created:** 2020/07/19<br>
**Last modified:** 2021/07/19<br>
**Description:** 
Example of a network to create bounding boxes and label, based on [Divam Gupta (2019)](https://divamgupta.com/image-segmentation/2019/06/06/deep-learning-semantic-segmentation-keras.html)

# A Beginner's guide to Deep Learning based Semantic Segmentation using Keras

## Path config

If you want the files to be copied to another folder within the same machine you are working on, by a source path other than remote.

In [1]:
PROJECT_SLUG = 'vigilancia_mascotas'
NAME = 'semantic_segmentation_test'
NUMBER = '0.0'


NOTEBOOK_NAME = f'{NUMBER}-{PROJECT_SLUG}-{NAME}.ipynb'

# COLAB
#DRIVE_MOUNT = '/drive'
#REMOTE_PATH = f'{DRIVE_MOUNT}/MyDrive/proyects/{PROJECT_SLUG}'
#LOCAL_PATH = '.'
#NOTEBOOK_PATH = f'{DRIVE_MOUNT}/MyDrive/Colab Notebooks/{NOTEBOOK_NAME}'

# LOCAL
REMOTE_PATH = '..'
LOCAL_PATH = '..'
DRIVE_MOUNT = ''
NOTEBOOK_PATH = f'G:\\Mi unidad\\Colab Notebooks\\{NOTEBOOK_NAME}'

In [2]:
if DRIVE_MOUNT:
    from google.colab import drive
    drive.mount(DRIVE_MOUNT)
    to_remote = True
else:
    to_remote = False

## Path functions

In [3]:
import sys

# It depends on where the library that comes with this package is stored.
sys.path.append(REMOTE_PATH)

In [6]:
from vigilancia_mascotas.utils.paths \
    import make_remote_copy_of_workspace_functions

In [7]:
local_dir, update_from_remote, update_to_remote, update_notebook = \
    make_remote_copy_of_workspace_functions(
        local_path=LOCAL_PATH,
        remote_path=REMOTE_PATH,
        notebook_path = NOTEBOOK_PATH
    )

## Implementation

We will be using Keras for building and training the segmentation models. First, install `keras_segmentation` which contains all the utilities required.

## Dataset

The first step in training our segmentation model is to prepare the dataset. We would need the input RGB images and the corresponding segmentation images. If you want to make your own dataset, a tool like [labelme](https://github.com/wkentaro/labelme) or [GIMP](https://www.gimp.org/) can be used to manually generate the ground truth segmentation masks.

Assign each class a unique ID. In the segmentation images, the pixel value should denote the class ID of the corresponding pixel. This is a common format used by most of the datasets and `keras_segmentation`. For the segmentation maps, do not use the `jpg` format as `jpg` is lossy and the pixel values might change. Use `bmp` or png format instead. And of course, the size of the input image and the segmentation image should be the same.

In the following example, pixel (0,0) is labeled as class 2, pixel (3,4) is labeled as class 1 and rest of the pixels are labeled as class 0.

In [14]:
from os import makedirs

In [15]:
ann_1_path = local_dir('data', 'processed', 'semantic_segmentation', 
                       'ann_1.png')

In [16]:
makedirs(ann_1_path.parent, exist_ok=True)

In [18]:
import cv2
import numpy as np

ann_img = np.zeros((30,30,3)).astype('uint8')
ann_img[ 3 , 4 ] = 1 # this would set the label of pixel 3,4 as 1
ann_img[ 0 , 0 ] = 2 # this would set the label of pixel 0,0 as 2

cv2.imwrite(str(ann_1_path), ann_img)

True

After generating the segmentation images, place them in the training/testing folder. Make separate folders for input images and the segmentation images. The file name of the input image and the corresponding segmentation image should be the same.

Refer to the format below :

```
dataset/
	train_images/
		- img0001.png
		- img0002.png
		- img0003.png
	train_segmentation/
		- img0001.png
		- img0002.png
		- img0003.png
	val_images/
		- img0004.png
		- img0005.png
		- img0006.png
	val_segmentation/
		- img0004.png
		- img0005.png
		- img0006.png

```

### Download and unzip the example dataset

You can refer to a sample dataset [here](https://github.com/divamgupta/image-segmentation-keras/tree/master/test/example_dataset).


For this tutorial we would be using a data-set which is already prepared. You can download it from [here](https://drive.google.com/file/d/0B0d9ZiqAgFkiOHR1NTJhWVJMNEU/view?usp=sharing).

In [25]:
import gdown

In [33]:
file_id = '0B0d9ZiqAgFkiOHR1NTJhWVJMNEU'
zip_path = local_dir('tmp', 'semantic_segmentation', 'dataset1.zip')
dataset_path = local_dir('data', 'raw', 'semantic_segmentation')

makedirs(zip_path.parent, exist_ok=True)
makedirs(dataset_path, exist_ok=True)

In [60]:
if not dataset_path.is_dir() and not zip_path.is_file():
    url_gdown = f'https://drive.google.com/u/0/uc?id={file_id}'
    gdown.download(url_gdown, str(zip_path), quiet=False)

In [61]:
import zipfile
if not dataset_path.is_dir():
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_path)

### Rename folders

In [36]:
from os import rename

In [62]:
dataset1_path = dataset_path.joinpath('dataset1')

train_images_path = dataset1_path.joinpath('train_images')
train_segmentation_path = dataset1_path.joinpath('train_segmentation')

val_images_path = dataset1_path.joinpath('val_images')
val_segmentation_path = dataset1_path.joinpath('val_segmentation')

old_train_images_path = dataset1_path.joinpath('images_prepped_train')
old_train_segmentation_path =dataset1_path.joinpath('annotations_prepped_train')

old_val_images_path = dataset1_path.joinpath('images_prepped_test')
old_val_segmentation_path = dataset1_path.joinpath('annotations_prepped_test')

In [63]:
for dst, src in zip(
    [train_images_path, train_segmentation_path, val_images_path,
    val_segmentation_path],
    [old_train_images_path, old_train_segmentation_path, old_val_images_path,
    old_val_segmentation_path],
):

    try:
        rename(src, dst)
    except Exception as e:
        print(e)

## Data Augmentation

If you have less number of training pairs, the results might not be good be because the model might overfit. We can increase the size of the dataset by applying random transformations on the images. We can change the color properties like hue, saturation, brightness, etc of the input images. We can also apply transformations such as rotation, scale, and flipping. For the transformations which change the location of the pixels, the segmentation image should also be transformed the same way.

![Example of image augmentation for segmentation](data:image/png;base64,)

_Example of image augmentation for segmentation [Gupta (2019)](https://divamgupta.com/image-segmentation/2019/06/06/deep-learning-semantic-segmentation-keras.html)_.

## Update local folders

In [8]:
update_from_remote()

There is no need to copy the remote files, as the remote repository does not exist or is empty.


## Update remote folders

In [ ]:
update_notebook(to_remote)

In [ ]:
update_to_remote()